In [ ]:
import cv2
import numpy as np

In [2]:
def measure_object(image_path, qr_code_size_cm, display=False, max_display_width=1600, max_display_height=1200):
    # Load the image
    image = cv2.imread(image_path)
    
    if image is None:
        raise ValueError(f"Image at path '{image_path}' could not be loaded.")

    # Convert to grayscale for easier processing
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # Blur the image to reduce noise
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)
    
    # Adaptive thresholding for better edge detection
    thresh = cv2.adaptiveThreshold(
        blurred, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
        cv2.THRESH_BINARY_INV, 11, 2
    )
    
    # Find contours
    contours, _ = cv2.findContours(
        thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE
    )
    
    if not contours:
        raise ValueError("No contours found in the image.")

    # Identify QR code contour based on aspect ratio and area
    qr_code_contour = None
    qr_candidates = []
    for contour in contours:
        area = cv2.contourArea(contour)
        if area < 100:  # Ignore small contours
            continue
        approx = cv2.approxPolyDP(
            contour, 0.02 * cv2.arcLength(contour, True), True
        )
        if len(approx) == 4:
            x, y, w, h = cv2.boundingRect(approx)
            aspect_ratio = w / float(h)
            if 0.9 <= aspect_ratio <= 1.1:  # Approximate square shape
                qr_candidates.append((area, contour))
    
    if not qr_candidates:
        raise ValueError("QR code not found in the image.")
    
    # Assume the largest square-like contour is the QR code
    qr_code_contour = max(qr_candidates, key=lambda x: x[0])[1]
    qr_x, qr_y, qr_w, qr_h = cv2.boundingRect(qr_code_contour)
    qr_code_width_pixels = (qr_w + qr_h) / 2  # Average width and height
    
    # Calculate scale in cm/pixel
    scale_cm_per_pixel = qr_code_size_cm / qr_code_width_pixels
    
    # Identify tree trunk contour based on area
    trunk_contours = [
        contour for contour in contours if contour is not qr_code_contour
    ]
    if not trunk_contours:
        raise ValueError("Tree trunk contour not found in the image.")
    
    # Assume the largest remaining contour is the tree trunk
    trunk_contour = max(trunk_contours, key=cv2.contourArea)
    trunk_x, trunk_y, trunk_w, trunk_h = cv2.boundingRect(trunk_contour)
    trunk_width_cm = trunk_w * scale_cm_per_pixel
    
    # Visualization
    if display:
        display_image = image.copy()
        cv2.rectangle(
            display_image, (trunk_x, trunk_y),
            (trunk_x + trunk_w, trunk_y + trunk_h), (0, 255, 0), 2
        )
        cv2.rectangle(
            display_image, (qr_x, qr_y),
            (qr_x + qr_w, qr_y + qr_h), (255, 0, 0), 2
        )
        
        # Resize image for display
        display_image = resize_image(
            display_image, max_width=max_display_width,
            max_height=max_display_height
        )
        
        cv2.imshow("Measured Image", display_image)
        cv2.waitKey(0)
        cv2.destroyAllWindows()
    
    return trunk_width_cm

def resize_image(image, max_width=1200, max_height=800):
    height, width = image.shape[:2]
    width_ratio = max_width / width
    height_ratio = max_height / height
    scaling_factor = min(width_ratio, height_ratio, 1)  # Ensure image is not scaled up
    new_width = int(width * scaling_factor)
    new_height = int(height * scaling_factor)
    resized_image = cv2.resize(image, (new_width, new_height), interpolation=cv2.INTER_AREA)
    return resized_image


def estimate_circumference(trunk_width_cm1, trunk_width_cm2):
    # Calculate semi-major and semi-minor axes
    a = trunk_width_cm1 / 2
    b = trunk_width_cm2 / 2
    
    # Ramanujan's approximation for ellipse circumference
    h = ((a - b) ** 2) / ((a + b) ** 2)
    circumference = np.pi * (a + b) * (1 + (3 * h) / (10 + np.sqrt(4 - 3 * h)))
    
    return circumference

In [3]:
# Paths to your images
image_path1 = "img1.jpg"
image_path2 = "img2.jpg"

# Known QR code size in centimeters
qr_code_size_cm = 20.0

try:
    # Measure the width of the tree trunk in both images
    trunk_width_cm1 = measure_object(
        image_path1, qr_code_size_cm, display=True
    )
    trunk_width_cm2 = measure_object(
        image_path2, qr_code_size_cm, display=True
    )
    
    # Estimate the circumference of the tree trunk
    circumference_cm = estimate_circumference(
        trunk_width_cm1, trunk_width_cm2
    )
    
    print(f"Estimated Circumference of the Tree Trunk: {circumference_cm:.2f} cm")
except ValueError as e:
    print(f"Error: {e}")

Estimated Circumference of the Tree Trunk: 120.37 cm
